# 1\) Etapas do Pré - Processamento

In [4801]:
import glob
import pandas as pd
import re
from pypdf import PdfReader

In [4802]:
diretorio_planilhas = "C:\\DataSets\\Projeto_Leitura_Documentos\\planilhas"

In [4803]:
def substituir(texto_atual,texto_novo,regex):
    return regex.sub(texto_novo,texto_atual)

# 2) Continuando a partir do acordao 01

In [4804]:
passo01 = pd.read_csv(f"{diretorio_planilhas}\\acordaos01.csv",index_col=[0])
passo01.head()

,indice,conteudo
19,19,stado do Acre. \nACÓRDÃO Ne 176 \nFeito \t : ...
127,127,Tribunal de Contas do Estado do Acre\nSecretar...
128,128,Tribunal de Contas do Estado do Acre\nSecretar...
129,129,Tribunal de Contas do Estado do Acre\nSecretar...
130,130,Tribunal de Contas do Estado do Acre\nSecretar...


# 3) Corrigindo indice (deixando sequêncial)
#### OBS.: 
#### Lembrando que anteriormente haviamos excluido registros com Nan e quebrou a sequência
#### a quebra de sequência no indice ocasiona diversos erros
#### por isso estamos fazendo este ajuste.
#### Também observar que deve começar de ZERO para facilotar o uso do FOR

In [4805]:
indices = []
n = -1
for i in passo01.index:
    n += 1
    indices.append(n)

extrutura_transitorio_indice = {
  'indice': indices
}
passo01.index = extrutura_transitorio_indice['indice']
passo01.drop('indice',axis='columns', inplace=True)
passo01.head()

,conteudo
0,stado do Acre. \nACÓRDÃO Ne 176 \nFeito \t : ...
1,Tribunal de Contas do Estado do Acre\nSecretar...
2,Tribunal de Contas do Estado do Acre\nSecretar...
3,Tribunal de Contas do Estado do Acre\nSecretar...
4,Tribunal de Contas do Estado do Acre\nSecretar...


# 4) Confirmando que não existe registros com Nan no Dataframe

In [4806]:
def informações_tabela(tabela):
    print(f"Quantidade total de registos {passo01.shape[0]} antes de apagar registros com NAN")
    total_com_na_conteudo = passo01['conteudo'].isna().sum()
    total_sem_na_conteudo = passo01['conteudo'].notna().sum()
    total_isnull = passo01['conteudo'].isnull().sum()

    print(f"Quantidade de registos sem NaN em conteudo: {total_sem_na_conteudo}")
    print(f"Quantidade de registos com NaN em conteudo: {total_com_na_conteudo}")
    print(f"Quantidade de registos com null em conteudo: {total_isnull}")

informações_tabela(passo01)

Quantidade total de registos 2454 antes de apagar registros com NAN
Quantidade de registos sem NaN em conteudo: 2454
Quantidade de registos com NaN em conteudo: 0
Quantidade de registos com null em conteudo: 0


# 5) Adicionando novas colunas "acordao" e "dia"

In [4807]:
# atualiza
passo01['acordao'] = ''
passo01['dia'] = ''
passo01.head()

,conteudo,acordao,dia
0,stado do Acre. \nACÓRDÃO Ne 176 \nFeito \t : ...,,
1,Tribunal de Contas do Estado do Acre\nSecretar...,,
2,Tribunal de Contas do Estado do Acre\nSecretar...,,
3,Tribunal de Contas do Estado do Acre\nSecretar...,,
4,Tribunal de Contas do Estado do Acre\nSecretar...,,


# 6) Fazendo pesquisa nos registros:

In [4808]:
def quantidade_ocorrencias(tabela,expressao_regular_a_procurar):
    resultado = []
    for i in tabela.index:
        previa = re.findall(expressao_regular_a_procurar,tabela.conteudo[i])
        if previa != []:
            resultado = resultado + previa
    return f"Encontrado {len(resultado)} vezes"



In [4809]:
quantidade_ocorrencias(passo01,"[0-9] [0-9]")

'Encontrado 1183 vezes'

In [4810]:
def substituir(texto_original,novo_texto,regex):
    return regex.sub(novo_texto,texto_original)

# 7) Fazendo pré-compilação do regex 
##### Objetivo: Para para ganhar desempenho

In [4811]:
regex_estado_do_acre = re.compile(r"^stado do acre\.")
regex_acordao = re.compile(r"a c ó r d ã o")
regex_pontos = re.compile(r"\.")
regex_tracos = re.compile(r"(-|-){1,}")
regex_acordam = re.compile(r" a c o r d a m ")
regex_espacos_em_branco_em_duplicidade = re.compile(r"( {2,})")
regex_tribunal_secretaria = re.compile(r"tribunal de contas do estado do acre(.|\n)secretaria das sessões")
regex_missao = re.compile(r"(missão)(:|)(.*)(social)")
regex_endereco = re.compile(r"(av ceará)(.|\n)*(govbr)")
regex_sala_das_sessoes = re.compile(r"sala das sessões do tribunal de contas do estado do acre")
regex_dia_sessao = re.compile(r"(rio branco )(.|)( acre)(,|)( [0-9][0-9] de | [0-9] de )(janeiro|fevereiro|março|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)( de )([0-9][0-9][0-9][0-9])(\.|)")
regex_virgula = re.compile(r"( ,|,)")

#Padronizando as datas
regex_janeiro = re.compile(r"( de janeiro de )")
regex_fevereiro = re.compile(r"( de fevereiro de )")
regex_marco = re.compile(r"( de março de )")
regex_abril = re.compile(r"( de abril de )")
regex_maio = re.compile(r"( de maio de | maio de )")
regex_junho = re.compile(r"( de junho de )")
regex_julho = re.compile(r"( de julho de )")
regex_agosto = re.compile(r"( de agosto de )")
regex_setembro = re.compile(r"( de setembro de )")
regex_outubro = re.compile(r"( de outubro de )")
regex_novembro = re.compile(r"( de novembro de )")
regex_dezembro = re.compile(r"( de dezembro de )")
regex_data = re.compile(r"(rio branco)( |, )([0-9][0-9])([0-9][0-9])([0-9][0-9][0-9][0-9])")
regex_ultimas_linhas = re.compile(r"(<fim?>)(.|\n){1,}")

#Retirando as pontuações
regex_pontuacao = re.compile(r"[^\w\s]")
regex_apagar_01 = re.compile(r"(tribunal de contas do estado do acre |secretaria das sessões | natureza do feito| feito | n )")
regex_numeros = re.compile(r"[0-9]")
regex_espacos_entre_numeros = re.compile(r"([0-9])( )([0-9])")
regex_numero_encostado_em_letras = re.compile(r"([0-9])([a-z])")
regex_espaco_entre_numeros = re.compile(r"(.)([0-9])( )([0-9])")

regex_numero_acordao = re.compile(r"(acórdão n |acórdão ni |acórdão n. |acórdão n º )")
regex_quebra_de_linha = re.compile(r"(\n){1,}")
regex_complemento_acordao = re.compile(r"(plen)(á|a)(riotceac)")
regex_tceac_ss = re.compile(r"(tribunal de contas do estado do acre)(.*)(secretaria das sessões)")
regex_acordao = re.compile(r"(a c ó r d ã o|a co r d ã o|a c o r d ã o)")
regex_ate_acordao = re.compile('^(.*)(br|)( |)(acórdão nº{1})')
regex_espacos_em_branco_em_duplicidade = re.compile(r"( {2,})")
regex_acordao_acordao = re.compile(r"(acórdão acórdão)")
regex_acordao_complemento = re.compile(r"(plenáriotceac|plenariotceac)")
regex_especiais = re.compile(r"[ºª]")
regex_nao_visuais = re.compile(r"(\r|\t){1,}")

regex_apagar_02 = re.compile("(acórdão |processo |tceacre |tceac |tce |tribunal de contas do estado do acre |secretaria das sessões |anexos |assunto )")
regex_uma_a_duas_letras = re.compile(" [a-z]{1,2} ")

# 8) Funções de substituições
#### 8.1) Faremos alterações para pegar o número do Acórdão (No início do documento)
#### 8.2) Faremos alterações para pegar a data (No final do documento)

In [4812]:
def substituicoes_relacionado_a_data(tabela):
    print('Aguarde', end = "")
    for i in tabela.index:
        #padronizando em letras minúsculas
        antes = tabela.conteudo[i].lower()
        
        depois = substituir(antes," ",regex_pontuacao)
        depois = substituir(depois," ",regex_quebra_de_linha)
        depois = substituir(depois," ",regex_tracos)
        
        depois = substituir(depois,"/01/",regex_janeiro)
        depois = substituir(depois,"/02/",regex_fevereiro)
        depois = substituir(depois,"/03/",regex_marco)
        depois = substituir(depois,"/04/",regex_abril)
        depois = substituir(depois,"/05/",regex_maio)
        depois = substituir(depois,"/06/",regex_junho)
        depois = substituir(depois,"/07/",regex_julho)
        depois = substituir(depois,"/08/",regex_agosto)
        depois = substituir(depois,"/09/",regex_setembro)
        depois = substituir(depois,"/10/",regex_outubro)
        depois = substituir(depois,"/11/",regex_novembro)
        depois = substituir(depois,"/12/",regex_dezembro)
        depois = substituir(depois,r"\2\3\4",regex_data)
        depois = substituir(depois," ",regex_espacos_em_branco_em_duplicidade)

        tabela.loc[i,'conteudo'] = depois

def substituicoes_relacionado_a_acordao(tabela):
    print('Aguarde', end = "")
    for i in tabela.index:
        #padronizando em letras minúsculas
        antes = tabela.conteudo[i].lower()
        
        depois = substituir(antes," ",regex_estado_do_acre)
        depois = substituir(depois," ",regex_complemento_acordao)
        depois = substituir(depois," ",regex_especiais)
        depois = substituir(depois," ",regex_nao_visuais)
        depois = substituir(depois,"acórdão",regex_acordao)
        depois = substituir(depois,"",regex_pontos)
        
        depois = substituir(depois," acordam ",regex_acordam)
        depois = substituir(depois," ",regex_tribunal_secretaria)
        depois = substituir(depois," ",regex_missao)
        depois = substituir(depois," ",regex_endereco)
        depois = substituir(depois," ",regex_sala_das_sessoes)
        depois = substituir(depois,r"\1\4\5\6\7\8",regex_dia_sessao)
        
        depois = substituir(depois," ",regex_virgula)
        
        
        depois = substituir(depois," ",regex_ultimas_linhas)
        depois = substituir(depois,"acórdão ",regex_numero_acordao)
        depois = substituir(depois,"",regex_pontuacao)
        depois = substituir(depois,"",regex_quebra_de_linha)
        
        depois = substituir(depois," ",regex_apagar_01)
        depois = substituir(depois," ",regex_espacos_em_branco_em_duplicidade)

        tabela.loc[i,'conteudo'] = depois



# 9) Adicionando COLUNAS "acordao" e "dia"

In [4813]:
substituicoes_relacionado_a_acordao(passo01)
passo01.head()

Aguarde

,conteudo,acordao,dia
0,acórdão 176 processo 2 57091tceacre relator c...,,
1,acórdão 7556natureza do processo 12906200901t...,,
2,acórdão 7557natureza do processo 14779201140t...,,
3,acórdão 7560natureza do processo 14884201101t...,,
4,acórdão 7564natureza do processo 14626201150t...,,


# 10) Adequações:
#### 10.1) Retirando Espaço Vazio que esteja entre números
#### 10.2) Separando números de letras

In [4814]:
conteudos = []
try:
    for texto in passo01['conteudo']:
        resposta = substituir(texto,r"\1\3",regex_espacos_entre_numeros)
        resposta = substituir(resposta,r"\1 \2",regex_numero_encostado_em_letras)
        conteudos.append(resposta)
except ValueError as numero_erro:
    print("Erro ao retirar espaço vazio entre números :", numero_erro)


extrutura_transitorio_conteudos = {
  'conteudo': conteudos
}

transitorio_conteudo = pd.DataFrame(extrutura_transitorio_conteudos)
passo01['conteudo'] = transitorio_conteudo
passo01.head()


,conteudo,acordao,dia
0,acórdão 176 processo 257091 tceacre relator c...,,
1,acórdão 7556 natureza do processo 12906200901...,,
2,acórdão 7557 natureza do processo 14779201140...,,
3,acórdão 7560 natureza do processo 14884201101...,,
4,acórdão 7564 natureza do processo 14626201150...,,


# 11) Pegando o número do Acórdão

In [4815]:
acordaos = []
for i in passo01.index:
    texto = passo01.conteudo[i]
    acordao = re.findall("(acórdão)( n. | n | )([0-9]{1,})( )",texto)
    if len(acordao) >= 1:
        acordaos.append(acordao[0][2]) 
    else:
        acordaos.append("00000000")

extrutura_transitorio_acordaos = {
  'acordao': acordaos
}

transitorio_acordaos = pd.DataFrame(extrutura_transitorio_acordaos)
passo01['acordao'] = transitorio_acordaos
passo01.head()

,conteudo,acordao,dia
0,acórdão 176 processo 257091 tceacre relator c...,176,
1,acórdão 7556 natureza do processo 12906200901...,7556,
2,acórdão 7557 natureza do processo 14779201140...,7557,
3,acórdão 7560 natureza do processo 14884201101...,7560,
4,acórdão 7564 natureza do processo 14626201150...,7564,


# 12) Analisando o que falta de Acórdãos

In [4816]:
def Retorna_registros_sem_acordao(tabela):
    resultado = []
    for indice , acordao in enumerate(tabela['acordao']):
        if acordao == '00000000':
            resultado.append(indice)
    return resultado

def Retorna_registros_com_acordao(tabela):
    resultado = []
    for indice , acordao in enumerate(tabela['acordao']):
        if acordao != '00000000':
            resultado.append(indice)
    return resultado

sem_acordao = Retorna_registros_sem_acordao(passo01)
com_acordao = Retorna_registros_com_acordao(passo01)
print(f'Foram encontrados {len(sem_acordao)} registros SEM acordao:')
print(sem_acordao)
print(f'Foram encontrados {len(com_acordao)} registros COM acordao:')
print(com_acordao)


Foram encontrados 20 registros SEM acordao:
[199, 334, 670, 791, 1020, 1132, 1193, 1194, 1195, 1196, 1197, 1199, 1201, 1208, 1209, 1227, 1345, 1369, 1519, 1571]
Foram encontrados 2434 registros COM acordao:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

# 13) Finalizar ultimos Acórdãos

In [4817]:
def analise_acordao(inicio_em):
    antigo = pd.read_csv(f"{diretorio_planilhas}\\transitorio01.csv",index_col=[0])
    proxima = inicio_em
    inicio = 0 + proxima
    final = 5 + proxima
    pagina_atual = sem_acordao[inicio:final]
    print('Registros sem a ACORDAO:\n==========\n')
    for indice in pagina_atual:
        print(f'\nindice {indice}')
        print(f'Antes = \n       {antigo["conteudo"][indice][0:300]}')
        print(f'Agora = \n       {passo01["conteudo"][indice][0:300]}')
        resultados = re.findall("acordao..........................",antigo["conteudo"][indice])
        print(f'Resultado Agora: {passo01["acordao"][indice]}')
        for i, acordao in enumerate(resultados):
            print(f'       {i} - {acordao[0:30]}')
    print('==================\n')

In [4818]:
for texto in sem_acordao:
    print(passo01['conteudo'][sem_acordao][0:100])

199      primeira câmara estados_municipiostransferenc...
334                                                      
670      segunda câmara acórdã o 0220142 câmara tceac ...
791     9 gtaudek amue do cada dit _der acõrdão 2973 p...
1020     gabinete da presidên cia portaria 3722017 o c...
1132    12 34 56 78 101112136 17 14101311141 710131013...
1193     processo tce 17158200556 parecer prévio 63320...
1194     gabinete do conselheiro v almir gomes ribeiro...
1195     gabinete do conselheiro valmir gomes ribeiro ...
1196     gabinete do conselheiro valmir go mes ribeiro...
1197     gabinete do conselheiro valmir gomes ribeiro ...
1199     gabinete do conselheiro valmir gomes ribeiro ...
1201     processo tce 16115201201 parecer prévio 67220...
1208     processo tce 16114201290 parecer prévio 66720...
1209     processo tce 17307201340 parecer prévio 66820...
1227     processo tce 18807201490 parecer prévio 66420...
1345     tribunal de contas do estado d o acre gabinet...
1369     proce

In [4819]:
analise_acordao(199)

Registros sem a ACORDAO:




# - - - - - - 

# 14) Adequações

In [4820]:
# Pré compilação
regex_janeiro_cortado = re.compile(r"janeir o")
regex_fevereiro_cortado = re.compile(r"(fevere iro|fevereir o)")
regex_marco_cortado = re.compile(r"març o")
regex_maio_cortado = re.compile(r"m aio")
regex_agosto_cortado = re.compile(r"agost o")
regex_outubro_cortado = re.compile(r"outubr o")
regex_novembro_cortado = re.compile(r"(novembr o|novem bro)")
regex_dezembro_cortado = re.compile(r"(dezem bro)")
regex_primeiro = re.compile(r"(1º)")
regex_rio_branco_acre = re.compile(r"rio brancoacre |rio branco ac ")
regex_data = re.compile(r"(rio branco acre )([0-9][0-9])([0-9][0-9])([0-9][0-9][0-9][0-9])")



conteudos = []
try:
    for texto in passo01['conteudo']:
        resposta = substituir(texto,r"janeiro",regex_janeiro_cortado)
        resposta = substituir(resposta,r"fevereiro",regex_fevereiro_cortado)
        resposta = substituir(resposta,r"março",regex_marco_cortado)
        resposta = substituir(resposta,r"maio",regex_maio_cortado)
        resposta = substituir(resposta,r"agosto",regex_agosto_cortado)
        resposta = substituir(resposta,r"outubro",regex_outubro_cortado)
        resposta = substituir(resposta,r"novembro",regex_novembro_cortado)
        resposta = substituir(resposta,r"dezembro",regex_dezembro_cortado)
        resposta = substituir(resposta,r"01",regex_primeiro)
        resposta = substituir(resposta,r"\2/\3/\4",regex_data)
        resposta = substituir(resposta,r"rio branco acre ",regex_rio_branco_acre)
        resposta = substituir(resposta," ",regex_espacos_em_branco_em_duplicidade)
        conteudos.append(resposta)
except ValueError as numero_erro:
    print("Erro ao retirar espaço vazio entre números :", numero_erro)


extrutura_transitorio_conteudos = {
  'conteudo': conteudos
}

transitorio_conteudo = pd.DataFrame(extrutura_transitorio_conteudos)
passo01['conteudo'] = transitorio_conteudo
passo01.head()

,conteudo,acordao,dia
0,acórdão 176 processo 257091 tceacre relator c...,176,
1,acórdão 7556 natureza do processo 12906200901...,7556,
2,acórdão 7557 natureza do processo 14779201140...,7557,
3,acórdão 7560 natureza do processo 14884201101...,7560,
4,acórdão 7564 natureza do processo 14626201150...,7564,


# 15) Pegando Data

In [4821]:
substituicoes_relacionado_a_data(passo01)
passo01.head()

Aguarde

,conteudo,acordao,dia
0,acórdão 176 processo 257091 tceacre relator c...,176,
1,acórdão 7556 natureza do processo 12906200901...,7556,
2,acórdão 7557 natureza do processo 14779201140...,7557,
3,acórdão 7560 natureza do processo 14884201101...,7560,
4,acórdão 7564 natureza do processo 14626201150...,7564,


In [4822]:

dias = []
try:
    for i, texto in enumerate(passo01['conteudo']):
        dia = re.findall("(rio branco)( acre |, | )([0-9][0-9]|[0-9])\/([0-9][0-9])\/([0-9][0-9][0-9][0-9])",texto)
        if len(dia) >= 1:
            print(f'{i} dia {dia[0][0]+dia[0][1]+dia[0][2]+"/"+dia[0][3]+"/"+dia[0][4]}')
            dias.append(dia[0][2]+'/'+dia[0][3]+'/'+dia[0][4])
        else:
            dias.append("00/00/0000")
except ValueError as numero_erro:
    print("Erro ao pegar a data :", numero_erro)


extrutura_transitorio_dias = {
  'dia': dias
}

transitorio_dia = pd.DataFrame(extrutura_transitorio_dias)
passo01['dia'] = transitorio_dia
passo01.head()

0 dia rio branco 02/04/1992
1 dia rio branco 02/02/2012
2 dia rio branco 02/02/2012
3 dia rio branco 02/02/2012
4 dia rio branco 02/02/2012
5 dia rio branco 02/02/2012
6 dia rio branco 02/02/2012
7 dia rio branco 02/02/2012
8 dia rio branco 02/02/2012
9 dia rio branco 02/02/2012
10 dia rio branco 02/02/2012
11 dia rio branco 02/02/2012
12 dia rio branco 04/10/2012
13 dia rio branco 09/02/2012
14 dia rio branco 14/06/2012
15 dia rio branco 09/02/2012
16 dia rio branco 14/06/2012
17 dia rio branco 09/02/2012
18 dia rio branco 14/06/2012
19 dia rio branco 09/02/2012
22 dia rio branco 09/02/2012
23 dia rio branco 09/02/2012
24 dia rio branco 09/02/2012
25 dia rio branco 09/02/2012
26 dia rio branco 09/02/2012
27 dia rio branco 09/02/2012
28 dia rio branco 21/06/2012
29 dia rio branco 21/06/2012
30 dia rio branco 21/06/2012
32 dia rio branco 01/03/2012
33 dia rio branco 01/03/2012
34 dia rio branco 01/03/2012
35 dia rio branco 04/07/2012
36 dia rio branco 12/07/2012
37 dia rio branco 08/03/

,conteudo,acordao,dia
0,acórdão 176 processo 257091 tceacre relator c...,176,02/04/1992
1,acórdão 7556 natureza do processo 12906200901...,7556,02/02/2012
2,acórdão 7557 natureza do processo 14779201140...,7557,02/02/2012
3,acórdão 7560 natureza do processo 14884201101...,7560,02/02/2012
4,acórdão 7564 natureza do processo 14626201150...,7564,02/02/2012


# = = = = = = 

# 16) Analisando o que falta de Datas

In [4823]:
def Retorna_registros_sem_data(tabela):
    resultado = []
    for indice , data in enumerate(tabela['dia']):
        if data == '00/00/0000':
            resultado.append(indice)
    return resultado

def Retorna_registros_com_data(tabela):
    resultado = []
    for indice , data in enumerate(tabela['dia']):
        if data != '00/00/0000':
            resultado.append(indice)
    return resultado

sem_data = Retorna_registros_sem_data(passo01)
com_data = Retorna_registros_com_data(passo01)
print(f'Foram encontrados {len(sem_data)} registros SEM DATA:')
print(sem_data)
print(f'Foram encontrados {len(com_data)} registros COM DATA:')
print(com_data)

Foram encontrados 610 registros SEM DATA:
[20, 21, 31, 49, 51, 53, 71, 73, 75, 77, 80, 82, 89, 98, 103, 111, 112, 113, 116, 121, 132, 135, 145, 146, 152, 161, 162, 165, 172, 177, 192, 193, 199, 200, 203, 213, 214, 216, 218, 221, 233, 234, 237, 238, 244, 245, 269, 275, 282, 289, 293, 297, 298, 299, 301, 310, 317, 321, 324, 330, 332, 333, 334, 337, 338, 339, 341, 343, 348, 350, 356, 358, 369, 370, 371, 372, 374, 379, 380, 387, 392, 396, 398, 400, 402, 407, 409, 411, 412, 413, 418, 422, 423, 425, 437, 438, 439, 440, 441, 445, 447, 448, 449, 450, 465, 470, 473, 474, 475, 476, 484, 492, 500, 505, 513, 514, 515, 518, 521, 523, 530, 534, 537, 551, 557, 562, 563, 570, 574, 575, 577, 582, 584, 586, 587, 589, 590, 598, 603, 608, 609, 614, 616, 622, 624, 625, 629, 634, 635, 636, 643, 645, 648, 650, 651, 653, 654, 655, 661, 672, 678, 679, 691, 694, 695, 696, 701, 702, 703, 711, 712, 713, 715, 716, 717, 718, 720, 724, 727, 728, 730, 731, 734, 736, 737, 743, 744, 747, 749, 750, 751, 753, 754, 756, 7

In [4824]:
passo01['dia'][31]

'00/00/0000'

# 17) Analise em registros que não consegui pegar a data

In [4825]:
def analise_registros_data():
    linhas = []
    antigo = pd.read_csv(f"{diretorio_planilhas}\\transitorio01.csv",index_col=[0])
    print('Registros sem a DATA:\n==========\n')
    for indice in sem_data:
        linhas.append(f'\nindice {indice}')
        resultados = re.findall("(rio branco......................)",antigo["conteudo"][indice])
        linhas.append(f'Antes = \n')
        for i, data in enumerate(resultados):
            linhas.append(f'       {i} - {data}')
        resultados = re.findall("(rio branco.......................)",passo01["conteudo"][indice])
        linhas.append(f'Agora = \n')
        for i, data in enumerate(resultados):
            linhas.append(f'       {i} - {data}')
    print('================\nVER RESPOSTA EM CSV RESULTADOS_SEM_DATA\n===============')

    extrutura_resultado = {
    'linha': linhas
    }
    transitorio_dia = pd.DataFrame(extrutura_resultado)
    transitorio_dia.to_csv(f"{diretorio_planilhas}\\resultados_sem_data.csv")

In [4826]:
analise_registros_data()

Registros sem a DATA:

VER RESPOSTA EM CSV RESULTADOS_SEM_DATA


In [4827]:
def Retorna_registros_sem_data(tabela):
    resultado = []
    for indice , dia in enumerate(tabela['dia']):
        if dia == '00/00/0000':
            resultado.append(indice)
    return resultado

def Retorna_registros_com_data(tabela):
    resultado = []
    for indice , dia in enumerate(tabela['dia']):
        if dia != '00/00/0000':
            resultado.append(indice)
    return resultado



sem_data = Retorna_registros_sem_data(passo01)
com_data = Retorna_registros_com_data(passo01)
print(f'Foram encontrados {len(sem_data)} registros SEM data:')
print(sem_data)
dia = passo01['dia'][20]
print(f'Exemplo: {dia}')
print(f'Foram encontrados {len(com_data)} registros COM data:')
print(com_data)
dia = passo01['dia'][0]
print(f'Exemplo: {dia}')


Foram encontrados 610 registros SEM data:
[20, 21, 31, 49, 51, 53, 71, 73, 75, 77, 80, 82, 89, 98, 103, 111, 112, 113, 116, 121, 132, 135, 145, 146, 152, 161, 162, 165, 172, 177, 192, 193, 199, 200, 203, 213, 214, 216, 218, 221, 233, 234, 237, 238, 244, 245, 269, 275, 282, 289, 293, 297, 298, 299, 301, 310, 317, 321, 324, 330, 332, 333, 334, 337, 338, 339, 341, 343, 348, 350, 356, 358, 369, 370, 371, 372, 374, 379, 380, 387, 392, 396, 398, 400, 402, 407, 409, 411, 412, 413, 418, 422, 423, 425, 437, 438, 439, 440, 441, 445, 447, 448, 449, 450, 465, 470, 473, 474, 475, 476, 484, 492, 500, 505, 513, 514, 515, 518, 521, 523, 530, 534, 537, 551, 557, 562, 563, 570, 574, 575, 577, 582, 584, 586, 587, 589, 590, 598, 603, 608, 609, 614, 616, 622, 624, 625, 629, 634, 635, 636, 643, 645, 648, 650, 651, 653, 654, 655, 661, 672, 678, 679, 691, 694, 695, 696, 701, 702, 703, 711, 712, 713, 715, 716, 717, 718, 720, 724, 727, 728, 730, 731, 734, 736, 737, 743, 744, 747, 749, 750, 751, 753, 754, 756, 7

In [4828]:
passo01['dia'][0]

'02/04/1992'

In [4829]:
def analise_data(inicio_em):
    antigo = pd.read_csv(f"{diretorio_planilhas}\\transitorio01.csv",index_col=[0])
    proxima = inicio_em
    inicio = 0 + proxima
    final = 5 + proxima
    pagina_atual = sem_data[inicio:final]
    print('Registros sem a data:\n==========\n')
    for indice in pagina_atual:
        print(f'\nindice {indice}')
        print(f'Antes = \n       {antigo["conteudo"][indice][-400:-1]}')
        print(f'Agora = \n       {passo01["conteudo"][indice][-400:-1]}')
        resultados = re.findall("rio branco............",passo01["conteudo"][indice])
        print(f'Resultado Agora:')
        for i, riobranco in enumerate(resultados):
            print(f'       {i} - {riobranco}')
    print('==================\n')

analise_data(0)

Registros sem a data:


indice 20
Antes = 
       nteiro neto principal responsávelà época 3dar ciência ao senhor governador do estado e ao presidente daassembléia legislativa do resultado desta decisão 4decidiuse por maioria nostermos do voto do conselheiro antônio jorge malheiro pela abertura de processodetomada de contas especial para apurar a movimentação do caixa noperíodo de 1º de janeiro a 31 de dezembro do exercício de 2005 para assim te
Agora = 
       teiro neto principal responsávelà época 3 dar ciência ao senhor governador do estado e ao presidente daassembléia legislativa do resultado desta decisão 4 decidiuse por maioria nostermos do voto do conselheiro antônio jorge malheiro pela abertura de processodetomada de contas especial para apurar a movimentação do caixa noperíodo de 1 de janeiro a 31 de dezembro do exercício de 2005 para assim te
Resultado Agora:

indice 21
Antes = 
       sificação dentre os incisos do art 2º do decretoestadual nº 6853122002 para que a 1ª igce 

# Com base no resultado acima fazer adequações no passo 14 !

# Tratamentos finais
### - Apagando Números

In [4830]:
def fazer_substituicoes_finais(tabela):
    print('Aguarde', end = "")
    for i in tabela.index:

        #padronizando em letras minúsculas
        antes = tabela.conteudo[i].lower()
        
        depois = substituir(antes," ",regex_numeros)
        depois = substituir(depois," ",regex_apagar_02)
        depois = substituir(depois," ",regex_uma_a_duas_letras)
        depois = substituir(depois," ",regex_espacos_em_branco_em_duplicidade)
        depois = substituir(depois," ",regex_nao_visuais)

        tabela.loc[i,'conteudo'] = depois
        print('.', end = "")

    print('\nTexto processado com sucesso !')

In [4831]:

#Substituições após pegar o acórdão e a data
fazer_substituicoes_finais(passo01)
passo01.head()


Aguarde.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

,conteudo,acordao,dia
0,relator conselheiro marciliano reis fleming v...,176,02/04/1992
1,natureza apenso prestação contas prefeitura m...,7556,02/02/2012
2,natureza tce prestação contas câmara municipa...,7557,02/02/2012
3,natureza anexos prestação contas empresa muni...,7560,02/02/2012
4,natureza tce relatório gestão fiscal prefeitu...,7564,02/02/2012


# Excluir registros que faltam número do Acórdão e data do julgamento

In [4832]:
set_sem_acordao = set(sem_acordao)
set_sem_data = set(sem_data)
indices_a_serem_excluidos = set_sem_acordao.union(set_sem_data)

In [4833]:
print(f'Apagando {len(sem_acordao)} registros sem número do acordao')
print(f'Apagando {len(sem_data)} registros sem data')
print(f'Total registros no DataFrame {passo01.shape[0]}')
print(f'Total a excluir {len(indices_a_serem_excluidos)} (sem repetições)')
print(f'Após a exclusão o Dataframe ficará com {passo01.shape[0] - len(indices_a_serem_excluidos)} linhas restantes')

Apagando 20 registros sem número do acordao
Apagando 610 registros sem data
Total registros no DataFrame 2454
Total a excluir 615 (sem repetições)
Após a exclusão o Dataframe ficará com 1839 linhas restantes


In [4834]:
lista_a_excluir = list(indices_a_serem_excluidos)
print(type(lista_a_excluir))
print(lista_a_excluir)

<class 'list'>
[20, 21, 31, 49, 51, 53, 71, 73, 75, 77, 80, 82, 89, 98, 103, 2154, 111, 112, 113, 116, 121, 132, 135, 145, 146, 152, 161, 162, 165, 172, 177, 2226, 2233, 192, 193, 199, 200, 2248, 203, 213, 214, 216, 2264, 218, 2268, 221, 233, 234, 237, 238, 244, 245, 2292, 2314, 269, 275, 282, 289, 293, 297, 298, 299, 301, 310, 2359, 317, 321, 2369, 324, 330, 332, 333, 334, 337, 338, 339, 341, 343, 348, 2397, 350, 2403, 356, 358, 2412, 369, 370, 371, 372, 374, 379, 380, 2430, 2432, 387, 392, 2441, 396, 398, 400, 402, 407, 409, 411, 412, 413, 418, 422, 423, 425, 437, 438, 439, 440, 441, 445, 447, 448, 449, 450, 465, 470, 473, 474, 475, 476, 484, 492, 500, 505, 513, 514, 515, 518, 521, 523, 530, 534, 537, 551, 557, 562, 563, 570, 574, 575, 577, 582, 584, 586, 587, 589, 590, 598, 603, 608, 609, 614, 616, 622, 624, 625, 629, 634, 635, 636, 643, 645, 648, 650, 651, 653, 654, 655, 661, 670, 672, 678, 679, 691, 694, 695, 696, 701, 702, 703, 711, 712, 713, 715, 716, 717, 718, 720, 724, 727, 72

In [4835]:
passo02 = passo01.drop(lista_a_excluir,axis=0)
print(f'Total registros no DataFrame {len(passo02.index)} após excluir os registros')

Total registros no DataFrame 1839 após excluir os registros


In [4836]:
indices = []
n = -1
for i in passo02.index:
    n += 1
    indices.append(n)

extrutura_transitorio_indice = {
  'indice': indices
}

passo02.index = extrutura_transitorio_indice['indice']
passo02.head()

,conteudo,acordao,dia
0,relator conselheiro marciliano reis fleming v...,176,02/04/1992
1,natureza apenso prestação contas prefeitura m...,7556,02/02/2012
2,natureza tce prestação contas câmara municipa...,7557,02/02/2012
3,natureza anexos prestação contas empresa muni...,7560,02/02/2012
4,natureza tce relatório gestão fiscal prefeitu...,7564,02/02/2012


In [4838]:
passo02.to_csv(f"{diretorio_planilhas}\\acordaos03.csv")


# Continuar no notebook Grupo_NLP_03.ipynb